# AE8803 Final Project

Author: Teo Wilkening

Class: AE8803 Machine Learning with Gaussian Processes

Due Date: 2024-05-03

This notebook has been built to show the real-time processing of
pressure readings received from a pressure sensor and pitot tube
attached to the Jetbot robot.

The goals of this project are:
1. Build a GP to predict the upcoming pressure readings.
2. Graphically display the GP predictions vs. observations.
3. Update the GP model at a specified interval based on observations.

Below is the data flow architecture that has been implemented to achieve the above
goals. Supporting files are attached in the submission and may be found on
GitHub: [ae8803/Final_Project](https://github.com/twilkening/ae8803/tree/main/Final_Project)

![diagram of the flow of data during processing](diagrams/data_flow.jpg)

## Start the Background Processes

In [9]:
import subprocess
import sys

In [10]:
def start_background_process(command, stdout_file, stderr_file):
    """Starts a background process and redirects its stdout and stderr to specified files."""
    # Open both files for writing
    stdout_f = open(stdout_file, "w")
    stderr_f = open(stderr_file, "w")

    process = subprocess.Popen(command, shell=True,
                                stdout=stdout_f,
                                stderr=stderr_f)
    return process, stdout_f, stderr_f

# DAQ process
if sys.platform == "win32":
    daq_proc, daq_stdout, daq_stderr = \
        start_background_process("python 1_DAQ_fake.py", "daq_stdout.log", "daq_stderr.log")
else:
    daq_proc, daq_stdout, daq_stderr = \
        start_background_process("python 1_DAQ.py", "daq_stdout.log", "daq_stderr.log")

# GP Update process
gp_proc, gp_stdout, gp_stderr = \
    start_background_process("python 1_GPUpdate.py", "gp_stdout.log", "gp_stderr.log")


## Start the Plotting

## Kill the Background Processes

In [11]:
def kill_process(process, stdout_f, stderr_f):
    """Kills a process given a subprocess.Popen object."""
    try:
        process.terminate()  # Sends SIGTERM
        process.wait(timeout=5)  # Wait up to 5 seconds for the process to gracefully exit
        stdout_f.close()
        stderr_f.close()
        print("stdout files closed")
    except subprocess.TimeoutExpired:
        process.kill()  # Force kill if not terminated after timeout
        process.wait()  # Wait for the process to be killed
        stdout_f.close()
        stderr_f.close()
        print("stdout files closed")
    except Exception as e:
        print(f"Failed to kill the process: {str(e)}")

kill_process(daq_proc, daq_stdout, daq_stderr)
kill_process(gp_proc, gp_stdout, gp_stderr)

stdout files closed
stdout files closed
